In [ ]:
"""
Author: Shaimaa K. El-Baklish
This file is under MIT License.
Link: https://github.com/shaimaa-elbaklish/funcMinimization/blob/main/LICENSE.md
"""

In [8]:
import numpy as np
import plotly.graph_objects as go

## Benchmark Multimodal Functions Available
| Function | Dimension | Bounds | Optimal Function Value |
| -------- | --------- | ------ | ---------------------- |
| $$ f_{1} = 4x_1^2 - 2.1x_1^4 + \frac{1}{3}x_1^6 + x_1 x_2 - 4x_2^2 + 4 x_2^4 $$ | 2 | [-5, 5] | -1.0316 |
| $$ f_{2} = (x_2 - \frac{5.1}{4\pi^2}x_1^2 + \frac{5}{\pi}x_1 -6)^2 +10(1 - \frac{1}{8\pi})\cos{x_1} + 10 $$ | 2 | [-5, 5] | 0.398 |
| $$ f_{3} = -\sum_{i=1}^{4} c_i exp(-\sum_{j=1}^{3} a_{ij}(x_j - p_{ij})^2) $$ | 3 | [1, 3] | -3.86 |
| $$ f_{4} = -\sum_{i=1}^{4} c_i exp(-\sum_{j=1}^{6} a_{ij}(x_j - p_{ij})^2) $$ | 6 | [0, 1] | -3.32 |
| $$ f_{5} = -\sum_{i=1}^{7} [(X - a_i)(X - a_i)^T + c_i]^{-1} $$ | 4 | [0, 10] | -10.4028 |

In [9]:
class Function:
    def __init__(self, x = None, n = 2, lb = np.array([-5, -5]), ub = np.array([5, 5])):
        self.n_x = n
        if x is not None:
            assert(x.shape[0] == self.n_x)
            self.fvalue = self.getFValue(x)
        self.x = x
        self.fvalue = None
        assert(lb.shape[0] == self.n_x)
        self.lb = lb
        assert(ub.shape[0] == self.n_x)
        self.ub = ub
        self.benchmark_selected = None
    
    def setBenchmarkFunction(self, f_name = "f1"):
        benchmarks = {
            "f1": [2, np.array([-5, -5]), np.array([5, 5])],
            "f2": [2, np.array([-5, -5]), np.array([5, 5])],
            "f3": [3, 1*np.ones(shape=(3,)), 3*np.ones(shape=(3,))],
            "f4": [6, np.zeros(shape=(6,)), 1*np.ones(shape=(6,))],
            "f5": [4, np.zeros(shape=(4,)), 10*np.ones(shape=(4,))]
        }
        self.benchmark_selected = f_name
        [self.n_x, self.lb, self.ub] = benchmarks.get(f_name, benchmarks.get("f1"))
    
    def isFeasible(self, x):
        return np.all(x >= self.lb) and np.all(x <= self.ub)
    
    def getFValue(self, x):
        if self.benchmark_selected is None:
            func_value = 4*x[0]**2 - 2.1*x[0]**4 + (x[0]**6)/3 + x[0]*x[1] - 4*x[1]**2 + 4*x[1]**4
            return func_value
        benchmarks_coeffs = {
            "f3": {"a": np.array([[3, 10, 30], [0.1, 10, 35], [3, 10, 30], [0.1, 10, 35]]),
                   "c": np.array([1, 1.2, 3, 3.2]),
                   "p": np.array([[0.3689, 0.117, 0.2673], [0.4699, 0.4387, 0.747], [0.1091, 0.8732, 0.5547], [0.03815, 0.5743, 0.8828]])},
            "f4": {"a": np.array([[10, 3, 17, 3.5, 1.7, 8], [0.05, 10, 17, 0.1, 8, 14], [3, 3.5, 1.7, 10, 17, 8], [17, 8, 0.05, 10, 0.1, 14]]),
                   "c": np.array([1, 1.2, 3, 3.2]),
                   "p": np.array([[0.1312, 0.1696, 0.5569, 0.0124, 0.8283, 0.5886], [0.2329, 0.4135, 0.8307, 0.3736, 0.1004, 0.9991], [0.2348, 0.1415, 0.3522, 0.2883, 0.3047, 0.6650], [0.4047, 0.8828, 0.8732, 0.5743, 0.1091, 0.0381]])},
            "f5": {"a": np.array([[4, 4, 4, 4], [1, 1, 1, 1], [8, 8, 8, 8], [6, 6, 6, 6], [3, 7, 3, 7], [2, 9, 2, 9], [5, 5, 3, 3], [8, 1, 8, 1], [6, 2, 6, 2], [7, 3.6, 7, 3.6]]),
                   "c": np.array([0.1, 0.2, 0.2, 0.4, 0.4, 0.6, 0.3, 0.7, 0.5, 0.5])}
        }
        benchmarks = {
            "f1": lambda z: 4*z[0]**2 - 2.1*z[0]**4 + (z[0]**6)/3 + z[0]*z[1] - 4*z[1]**2 + 4*z[1]**4,
            "f2": lambda z: (z[1] - (5.1/(4*np.pi**2))*z[0]**2 + (5/np.pi)*z[0] -6)**2 + 10*(1 - (1/(8*np.pi)))*np.cos(z[0]) + 10,
            "f3": lambda z: -np.sum(benchmarks_coeffs["f3"]["c"] * np.exp(-np.sum(list(map(lambda ai, pi: ai*(z - pi)**2, benchmarks_coeffs["f3"]["a"], benchmarks_coeffs["f3"]["p"])), axis=1))),
            "f4": lambda z: -np.sum(benchmarks_coeffs["f4"]["c"] * np.exp(-np.sum(list(map(lambda ai, pi: ai*(z - pi)**2, benchmarks_coeffs["f4"]["a"], benchmarks_coeffs["f4"]["p"])), axis=1))),
            "f5": lambda z: -np.sum(list(map(lambda ai, ci: 1/((z - ai) @ (z - ai).T + ci), benchmarks_coeffs["f5"]["a"], benchmarks_coeffs["f5"]["c"])))
        }
        func_value = benchmarks.get(self.benchmark_selected)(x)
        return func_value
    
    def initRandomSoln(self):
        self.x = np.random.rand(self.n_x) * (self.ub - self.lb) + self.lb
        assert(self.isFeasible(self.x))
        self.fvalue = self.getFValue(self.x)
    
    def getNeighbourSoln(self):
        r = np.random.rand(self.n_x)
        x_new = self.x + r * (self.ub - self.x) + (1 - r) * (self.lb - self.x)
        assert(self.isFeasible(x_new))
        return x_new

In [21]:
class GeneticAlgorithm:
    def __init__(self, problem, n_pop = 50, max_iter = 100, p_elite = 0.1, p_crossover = 0.8, p_mutation = 0.1, 
                        parents_selection = "Random", tournament_size = 5, mutation_selection = "Worst", survivors_selection = "Fitness"):
        self.problem = problem
        self.n_pop = n_pop
        self.max_iter = max_iter
        self.p_elite = p_elite
        self.p_crossover = p_crossover
        self.p_mutation = p_mutation
        self.parents_selection = parents_selection
        self.tournament_size = tournament_size if tournament_size < n_pop else n_pop
        self.mutation_selection = mutation_selection
        self.survivors_selection = survivors_selection
        self.gen_sols = None
        self.gen_fvalues = None
        self.gen_ages = None
        self.best_sols = None
        self.best_fvalues = None
    
    def initRandomPopulation(self):
        self.gen_sols = []
        self.gen_fvalues = []
        self.gen_ages = []
        self.best_sols = []
        self.best_fvalues = []
        for _ in range(self.n_pop):
            self.problem.initRandomSoln()
            new_sol = self.problem.x
            new_fvalue = self.problem.fvalue
            self.gen_sols.append(new_sol)
            self.gen_fvalues.append(new_fvalue)
            self.gen_ages.append(0)
            if len(self.best_sols) == 0:
                self.best_sols.append(new_sol)
                self.best_fvalues.append(new_fvalue)
            elif (new_fvalue < self.best_fvalues[0]):
                self.best_sols[0], self.best_fvalues[0] = new_sol, new_fvalue
    
    def selectParents(self, numParents, criteria):
        gen_probs = 1 / (1 + np.square(self.gen_fvalues))
        gen_probs = gen_probs / sum(gen_probs)
        lambda_rank = 1.5 # (between 1 and 2) offspring created by best individual
        gen_ranks = list(map(lambda i: np.argwhere(np.argsort(self.gen_fvalues) == i)[0,0], np.arange(self.n_pop)))
        gen_ranks = ((2-lambda_rank) + np.divide(gen_ranks, self.n_pop-1)*(2*lambda_rank-2)) / self.n_pop
        selection_criteria = {
            "Random": lambda n: np.random.choice(self.n_pop, size=(n,), replace=False),
            "RouletteWheel": lambda n: np.random.choice(self.n_pop, size=(n,), replace=True, p=gen_probs),
            "SUS": lambda n: np.random.choice(self.n_pop, size=(n,), replace=False, p=gen_probs),
            "Rank": lambda n: np.random.choice(self.n_pop, size=(n,), replace=False, p=gen_ranks),
            "Tournament": lambda n: np.array([np.amin(list(map(lambda i: [self.gen_fvalues[i], i], 
                                                                np.random.choice(self.n_pop, size=(self.tournament_size,), replace=False))), 
                                                axis=0)[1] for _ in range(n)], dtype=int),
            "Worst": lambda n: np.argsort(self.gen_fvalues)[self.n_pop-n:]
        }
        parents_idx = selection_criteria.get(criteria, selection_criteria["Random"])(numParents)
        return parents_idx

    def crossover(self, p1_idx, p2_idx):
        # Whole Arithmetic Combination
        alpha = np.random.rand() * (0.9 - 0.7) + 0.7
        child1 = alpha * self.gen_sols[p1_idx] + (1 - alpha) * self.gen_sols[p2_idx]
        child2 = (1 - alpha) * self.gen_sols[p1_idx] + alpha * self.gen_sols[p2_idx]
        return child1, child2

    def mutation(self, p_idx):
        # Random noise
        r = np.random.rand(self.problem.n_x)
        child = self.gen_sols[p_idx] + r * (self.problem.ub - self.gen_sols[p_idx]) + (1 - r) * (self.problem.lb - self.gen_sols[p_idx])
        return child
    
    def selectSurvivors(self, numSurvivors, criteria):
        selection_criteria = {
            "Age": lambda n: np.argsort(self.gen_ages)[:n],
            "Fitness": lambda n: np.argsort(self.gen_fvalues)[:n]
        }
        survivors_idx = selection_criteria.get(criteria, selection_criteria["Fitness"])(numSurvivors)
        return survivors_idx

    def perform_algorithm(self):
        self.initRandomPopulation()
        print("Best Initial Solution ", self.best_fvalues[0])
        n_crossovers = int(np.ceil(self.p_crossover * self.n_pop / 2))
        n_mutations = int(self.p_mutation * self.n_pop)
        n_elite = int(self.p_elite * self.n_pop)
        n_survivors = self.n_pop - int(self.p_crossover*self.n_pop) - n_mutations - n_elite
        for _ in range(self.max_iter):
            # Crossover and Parents Selection
            parents_idx = self.selectParents(numParents=n_crossovers*2, criteria=self.parents_selection)
            new_gen_sols = []
            new_gen_fvalues = []
            new_gen_ages = []
            for i in range(0, n_crossovers*2, 2):
                [ch1, ch2] = self.crossover(parents_idx[i], parents_idx[i+1])
                new_gen_sols.append(ch1)
                new_gen_fvalues.append(self.problem.getFValue(ch1))
                new_gen_ages.append(0)
                if len(new_gen_sols) == int(self.p_crossover * self.n_pop):
                    break
                new_gen_sols.append(ch2)
                new_gen_fvalues.append(self.problem.getFValue(ch2))
                new_gen_ages.append(0)
            # Mutation and Parents Selection
            parents_idx = self.selectParents(numParents=n_mutations, criteria=self.mutation_selection)
            for i in range(n_mutations):
                ch = self.mutation(parents_idx[i])
                new_gen_sols.append(ch)
                new_gen_fvalues.append(self.problem.getFValue(ch))
                new_gen_ages.append(0)
            # Elite Members
            elite_idx = self.selectSurvivors(numSurvivors=n_elite, criteria="Fitness")
            for i in range(n_elite):
                new_gen_sols.append(self.gen_sols[elite_idx[i]])
                new_gen_fvalues.append(self.gen_fvalues[elite_idx[i]])
                new_gen_ages.append(self.gen_ages[elite_idx[i]]+1)
            # Survivors (if any)
            survivors_idx = self.selectSurvivors(numSurvivors=n_survivors, criteria=self.survivors_selection)
            for i in range(n_survivors):
                new_gen_sols.append(self.gen_sols[survivors_idx[i]])
                new_gen_fvalues.append(self.gen_fvalues[survivors_idx[i]])
                new_gen_ages.append(self.gen_ages[survivors_idx[i]]+1)
            assert(len(new_gen_sols) == self.n_pop)
            assert(len(new_gen_fvalues) == self.n_pop)
            assert(len(new_gen_ages) == self.n_pop)
            # New generation becomes current one
            self.gen_sols = new_gen_sols
            self.gen_fvalues = new_gen_fvalues
            self.gen_ages = new_gen_ages
            # update best solution reached so far
            best_idx = np.argmin(self.gen_fvalues)
            if self.gen_fvalues[best_idx] < self.best_fvalues[-1]:
                self.best_sols.append(self.gen_sols[best_idx])
                self.best_fvalues.append(self.gen_fvalues[best_idx])
            else:
                self.best_sols.append(self.best_sols[-1])
                self.best_fvalues.append(self.best_fvalues[-1])
    
    def visualize(self):
        # convergence plot
        fig1 = go.Figure(data=go.Scatter(x=np.arange(0, self.max_iter), y=self.best_fvalues, mode="lines"))
        fig1.update_layout(
            title="Convergence Plot",
            xaxis_title="Iteration Number",
            yaxis_title="Fitness Value of Best So Far"
        )
        fig1.show()
        pass


In [30]:
problem = Function()
problem.setBenchmarkFunction(f_name="f2")
GA = GeneticAlgorithm(problem, n_pop = 50, max_iter=100, p_elite=0.1, p_crossover=0.7, p_mutation=0.1, 
                                parents_selection="SUS", tournament_size = 20, mutation_selection = "Worst", survivors_selection = "Age")
GA.perform_algorithm()
print(GA.best_sols[-1])
print(GA.best_fvalues[-1])
GA.visualize()

Best Initial Solution  2.149705174772726
[3.14249292 2.27470172]
0.39789141182892784
